In [ ]:
"""IMPORT EVERYTHING.. """
# Data manipulation
import pandas as pd
import numpy as np

# Quantitative analysis / indicators
import talib
import backtrader as bt

# # Machine learning / statistics
# import scipy
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression

# # Visualization
# import matplotlib.pyplot as plt
# import seaborn as sns
import mplfinance as mpf

# # Jupyter Notebook magic for inline plots
# %matplotlib inline


In [ ]:
df = pd.read_parquet("../Src/filtered.XAUUSD-M1.parquet", engine="pyarrow")

# print(df.head(3))

In [ ]:
from tafm import EnhancedTechnicalAnalyzer, create_analyzer, IndicatorConfig

df.rename(columns={
    'OPEN': 'Open',
    'HIGH': 'High',
    'LOW': 'Low',
    'CLOSE': 'Close',
    'VOL': 'Volume'
}, inplace=True)

def example_basic_indicators(df):
    """Example: Adding technical indicators"""

    # Initialize analyzer
    analyzer = create_analyzer(df)

    # Add indicators using fluent interface
    analyzer.add_indicator(
        IndicatorConfig(name='EMA', period=21)
    ).add_indicator(
        IndicatorConfig(name='EMA', period=50)
    ).add_indicator(
        IndicatorConfig(name='RSI', period=14)
    ).add_indicator(
        IndicatorConfig(name='MACD', fast_period=12, slow_period=26, signal_period=9)
    )

    analyzer.crossed_down('EMA_21', 'EMA_50', 'Cross_e2150')

    # View results
    print("DataFrame with indicators:")
    print(analyzer.df[['Close', 'EMA_21', 'EMA_50', 'RSI_14', 'MACD']].tail())

    # Get summary
    print("\nOperations Summary:")
    print(analyzer.summary())

    return analyzer

signal = example_basic_indicators(df)

In [ ]:
signal.df[signal.df['Cross_e2150'] == 1]

In [ ]:
from resampler import OHLCVResampler

# Resample to 1 hour - all columns preserved
resampler = OHLCVResampler(default_agg='last')


result = resampler.resample(signal.df, '1min', '2024-11-01', '2024-11-02')


result.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
}, inplace=True)



print(f"\nResampled shape: {result.shape}")

Cross_e2150 = result[result['Cross_e2150']==1].index

vline_series = pd.Series(np.nan, index=result.index)
vline_series.loc[Cross_e2150] = 1

result

In [ ]:

from ChartterX5 import Chartter

chartter = Chartter(config={
    'chart_type': 'candle',
    'style': 'charles',
    'volume': False
})

chartter._setup_tradingview_theme()

cnf = {
        'title': 'NONE',
        'style': chartter.tradingview_style,
        'volume': False,
        'show_nontrading': False,
        'datetime_format': '%Y-%m-%d',
        'xlabel': '',
        'ylabel': '',
        'xrotation': 0
    }
# result.head(3)

In [ ]:
# Create base chart
fig, axes =chartter.plot(
    result,
    addplot=[
        mpf.make_addplot(result['RSI_14'], panel=1, color='purple', ylabel='RSI'),
        mpf.make_addplot(result['EMA_21'], panel=0, color='white', ylabel='RSI'),
        mpf.make_addplot(result['EMA_50'], panel=0, color='white', ylabel='RSI'),
        mpf.make_addplot(vline_series,panel=1,type='scatter',markersize=20,color='orange')
    ],
    config=cnf,
    vlines = {
        'vlines': Cross_e2150.to_list(),
        'colors': 'orange',
        'linewidths': 0.9,
        'linestyle': '--',
        'alpha': 0.7
    },
    returnfig=True
)